In [1]:
from my_packages import signal_elaboration as s
from my_packages.utils import probes, probes_walk, HandlePaths
from my_packages.directory_data import  GetCoordinates, make_generator, make_all_generators, RootBatchGen
from my_packages.my_hdf5 import explore_library, see_groups_and_datasets
from my_packages.signal_elaboration import read_csv


import os
import pandas
import numpy as np
from datetime import datetime
import scipy.signal
import scipy.io
import pickle
import h5py

In [2]:
from pprint import pprint


# exploring the NAS
# you can explore the NAS using cmd line tools:

print("we have the following probes", probes)
print("the structure in which the files are saved is: ")
pprint(probes_walk)

external_drive = r"E:\\"

# obtain all possible paths from the json-like structure
data_paths = HandlePaths(base_path=external_drive)(probes_walk)
probe_paths = {probe: HandlePaths(base_path=os.path.join(external_drive,probe))(probes_walk[probe]) for probe in probes}
all_paths = HandlePaths(base_path=external_drive)(probes_walk) 
pprint(probe_paths)  
pprint(all_paths)


we have the following probes ['xfb31', 'XFE_04s', 'XFR31']
the structure in which the files are saved is: 
{'XFE_04s': {'r18': {}, 'r18_o': {}},
 'XFR31': {'r18_o': {'along_x': {}, 'along_y': {}}},
 'xfb31': {'r18': {}}}
{'XFE_04s': ['E:\\\\XFE_04s/r18', 'E:\\\\XFE_04s/r18_o'],
 'XFR31': ['E:\\\\XFR31/r18_o/along_x', 'E:\\\\XFR31/r18_o/along_y'],
 'xfb31': ['E:\\\\xfb31/r18']}
['E:\\\\/xfb31/r18',
 'E:\\\\/XFE_04s/r18',
 'E:\\\\/XFE_04s/r18_o',
 'E:\\\\/XFR31/r18_o/along_x',
 'E:\\\\/XFR31/r18_o/along_y']


In [3]:


## create an object for the probes that can handle all the measurements for each probe
file = "measurements.h5"
generators = make_all_generators(file, return_fullpaths=True)
              


In [4]:
pprint(generators)

#next(generators["XFE_04s"])

group = "XFE_04s"

gen = generators["XFE_04s"]

f = h5py.File(file, "r")

gr = f[group]
pprint(dict(gr.attrs))

print(list(gr["coordinates"].keys()))

x = np.array(gr["coordinates"]["x_coordinates"])
y = np.array(gr["coordinates"]["y_coordinates"])

print(gr.attrs['measurement_path'])


batch = next(gen)

f.close()



{'XFE_04s': <my_packages.directory_data.RootBatchGen object at 0x000001E305D70190>,
 'XFR31_along_x': <my_packages.directory_data.RootBatchGen object at 0x000001E346DAB7F0>,
 'XFR31_along_y': <my_packages.directory_data.RootBatchGen object at 0x000001E305D018E0>,
 'xfb31': <my_packages.directory_data.RootBatchGen object at 0x000001E305D2D3D0>}
{'creation date': '21/06/2023 21:01:18',
 'measurement_path': 'E:/XFE_04s/r18_o',
 'probe': 'XFE_04s'}
['measurement_points', 'x_coordinates', 'y_coordinates']
E:/XFE_04s/r18_o


In [7]:
gen.current_batch

['x86000.00y20500.001.csv',
 'x86000.00y20500.002.csv',
 'x86000.00y20500.003.csv',
 'x86000.00y20500.004.csv',
 'x86000.00y20500.005.csv',
 'x86000.00y20500.006.csv',
 'x86000.00y20500.007.csv',
 'x86000.00y20500.008.csv',
 'x86000.00y20500.009.csv',
 'x86000.00y20500.0010.csv',
 'x86000.00y20500.0011.csv',
 'x86000.00y20500.0012.csv',
 'x86000.00y20500.0013.csv',
 'x86000.00y20500.0014.csv',
 'x86000.00y20500.0015.csv',
 'x86000.00y20500.0016.csv',
 'x86000.00y20500.0017.csv',
 'x86000.00y20500.0018.csv',
 'x86000.00y20500.0019.csv',
 'x86000.00y20500.0020.csv',
 'x86000.00y20500.0021.csv',
 'x86000.00y20500.0022.csv',
 'x86000.00y20500.0023.csv',
 'x86000.00y20500.0024.csv',
 'x86000.00y20500.0025.csv',
 'x86000.00y20500.0026.csv',
 'x86000.00y20500.0027.csv',
 'x86000.00y20500.0028.csv',
 'x86000.00y20500.0029.csv',
 'x86000.00y20500.0030.csv',
 'x86000.00y20500.0031.csv',
 'x86000.00y20500.0032.csv',
 'x86000.00y20500.0033.csv',
 'x86000.00y20500.0034.csv',
 'x86000.00y20500.0035.

In [8]:
import pandas as pd
f1 = batch[0]
r = np.array(list(read_csv(f1)))

print(r.shape)

rr = np.rec.fromarrays(list(r.transpose()), dtype=[("time", "f"), ("amplitude", "f")])

df = pd.DataFrame(rr)

(100000, 2)


In [9]:
from tqdm import tqdm

from my_packages.my_hdf5 import *

fp = "test_batch.h5"

def measurements2dataset(library, batch_generator, dataset_name, compression_level=4):
    number_of_batches = len(batch_generator)
    datapoints = int(100e3)
    batch_size = batch_generator.batch_size
    # batch_size = batch_generator.batch_size

    dataset_shape = (batch_size, number_of_batches, 2, datapoints)

    dataset_attr = dict(
        observation_interval = "10us",
        number_of_points = datapoints,
        max_frequency_resolution = 1/datapoints, 
        structure_of_numpy_array = ("batch_size", "number_of_batches", "time, amplitude", "datapoints") 
    )

    if not exists(library):
        build_hdf5(name=library)

    with h5py.File(library, "a") as f:
        dataset = f.require_dataset(
            dataset_name, 
            shape=dataset_shape, 
            dtype="float64", 
            chunks=(batch_size, 1, 2, datapoints), 
            compression=compression_level
            )
        for batch in tqdm(batch_generator):
            dataset[:, batch_generator.number_of_batches_done, :, : ] = get_numpy_from_batch(batch)


    

    

def get_numpy_from_batch(batch):
    readings = [list(read_csv(csv_file_path)) for csv_file_path in batch]

    # I must transpose so that the shape has the number of points in the last position and 2 - ie time and amplitude - in 
    # second to last place
    
    return np.array(readings).transpose(0,2,1)

    

In [10]:
measurements2dataset("test_batch", gen, "test1")

  1%|          | 18/2166 [04:24<8:45:08, 14.67s/it]


KeyboardInterrupt: 

In [11]:
with h5py.File("test_batch", "r") as f:
    arr = f["test1"][:, 0, :, :]

In [15]:
arr.shape

(50, 2, 100000)

AttributeError: 'list' object has no attribute 'batch_size'

In [74]:
with h5py.File(fp, "a") as f:
    f["batch"][:,:,:50000] = np.ones((50,2,50000))

In [9]:
from my_packages.my_hdf5 import *
from pprint import pprint

print(get_all_h5())
pprint(see_groups_and_datasets("measurements.h5"))
pprint(see_groups_and_datasets("measurements.h5", "XFE_04s"))

['xfb31.h5', 'measurements.h5']
{'dataset_keys': ['measurement_points'],
 'group_keys': ['XFE_04s', 'XFR31', 'xfb31']}
{'dataset_keys': [], 'group_keys': []}


In [95]:
def save_measurement_info(library, dh, probes, measurement_info={}, group_info={}):
    path = dh.path
    probe = get_probe_from_path(probes, dh.path)
    xcoord = dh.coordinates["x"]; ycoord = dh.coordinates["y"] 
    if not exists(library):
        build_hdf5(name=library, groups=[probe])
    
    if not group_exist(library, probe):
        add_group(library, probe, **group_info)

    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    # open the group
    with h5py.File(library, "a") as f:
        g = f[probe]
        # create the dataset
        # require_dataset is the same as create_dataset. However, if the dataset already exists it does not overwirte.

        #check if the coordinate group already exists
        group_keys = [key for key, items in g.items() if isinstance(items, h5py.Group)]
        print(group_keys)
        if "coordinates" in group_keys:
            res = input("type y to overwrite")

            if res != "y":
                return 
            else:
                del g["coordinates"]

       

        coord_gr = g.create_group("coordinates")
        coord_gr.attrs["creation date"]= dt_string
        coord_gr.attrs["measurement_path"] = path
        coord_gr.attrs["description"] = \
        "These coordinates were obtained as the coordinates that appear atleast once among the \
        measurement points as found in the names of the csv files"

        x_ds=coord_gr.require_dataset("x_coordinates", shape=xcoord.shape, dtype=np.float32, data=xcoord)
        y_ds=coord_gr.require_dataset("y_coordinates", shape=ycoord.shape, dtype=np.float32, data=ycoord)
        points = coord_gr.require_dataset("measurement_points", shape=dh.points.shape, dtype=np.float32, data=dh.points)



    
def get_probe_from_path(probes, path):
    probe_ = [p for p in probes if p in path]
    # check there is one element in the probe list
    try:
        probe = (lambda x: x)(*probe_)
    except:
        raise("probe length is ", len(probe_))
    return probe

save_measurement_info("measurements.h5", dh, probes)


['coordinates']


In [75]:
path= "measurements.h5"
pprint(see_groups_and_datasets(path))
print(group_exist(path, "test"))

remove_group(path, "test")
add_group(path, "test", description = "this group is a test", owner="tomas")

{'dataset_keys': ['measurement_points'],
 'group_keys': ['XFE_04s', 'XFR31', 'test', 'xfb31']}
True
['XFE_04s', 'XFR31', 'test', 'xfb31']


In [8]:
path = "measurements.h5"


def explore_library(path, recursive=True):
    def printall(name, obj):
        print("NAME: {:^30}".format(name))
        print("Type: {:^20}".format(f"GROUP - Subgroups: {list(obj.keys())}" if isinstance(obj, h5py.Group) else "DATASET"))
        print("Parent Path: {:<10}".format(obj.parent.name))
        print("Attributes: ")
        pprint(dict(obj.attrs))
        if isinstance(obj, h5py.Dataset):
            print("shape: ", obj.shape, "____ dtype: ", obj.dtype) 
        print("\n\n\n")



    with h5py.File(path, "r") as f:
        if recursive:
            f.visititems(printall)
        else:
            for name, obj in f.items():
                printall(name, obj)

In [10]:
explore_library("measurements.h5", recursive=True)

NAME:             xfb31             
Type: GROUP - Subgroups: ['coordinates']
Parent Path: /         
Attributes: 
{}




NAME:       xfb31/coordinates       
Type: GROUP - Subgroups: ['measurement_points', 'x_coordinates', 'y_coordinates']
Parent Path: /xfb31    
Attributes: 
{'creation date': '14/09/2022 21:26:36',
 'description': 'These coordinates were obtained as the coordinates that '
                'appear atleast once among the         measurement points as '
                'found in the names of the csv files',
 'measurement_path': '/NAS/xfb31/r18'}




NAME: xfb31/coordinates/measurement_points
Type:       DATASET       
Parent Path: /xfb31/coordinates
Attributes: 
{}
shape:  (2166,) ____ dtype:  float32




NAME: xfb31/coordinates/x_coordinates
Type:       DATASET       
Parent Path: /xfb31/coordinates
Attributes: 
{}
shape:  (57,) ____ dtype:  float32




NAME: xfb31/coordinates/y_coordinates
Type:       DATASET       
Parent Path: /xfb31/coordinates
Attributes: 
{}
shape

In [71]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 14/09/2022 17:13:29


In [62]:
f = h5py.File("measurements.h5", "r")

for k, obj in f.items():
    if isinstance(obj, h5py.Dataset):
        print(obj.shape)

f.close()


(2166, 2)


In [73]:
x = dh.coordinates["x"]
y = dh.coordinates["y"]

cc = np.rec.fromarrays([dh.points[:,0]/1e3, dh.points[:,1]/1e3], dtype=[("x", "float16"), ("y", "float16")])

In [74]:
import pandas as pd

pd.DataFrame(cc)

,x,y
0,134.0,34.5
1,88.0,64.5
2,124.0,50.5
3,148.0,64.5
4,172.0,44.5
...,...,...
2161,182.0,34.5
2162,122.0,70.5
2163,182.0,32.5
2164,178.0,46.5


In [70]:
pip install pandas

     |████████████████████████████████| 11.3 MB 3.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
